In [76]:
import numpy as np
import pandas as pd
import random
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [77]:
def get_users(line):
    row, col = line.split("_")
    row = row.replace("r", "")
    col = col.replace("c", "")
    return int(row)

In [78]:
def get_items(line):
    row, col = line.split("_")
    row = row.replace("r", "")
    col = col.replace("c", "")
    return int(col)

In [79]:
data = pd.read_csv('data_train.csv')

In [80]:
data.head()

,Id,Prediction
0,r44_c1,4
1,r61_c1,3
2,r67_c1,4
3,r72_c1,3
4,r86_c1,5


In [81]:
data['userID'] = data['Id'].apply(get_users)
data['itemID'] = data['Id'].apply(get_items)
data = data.drop('Id', axis=1)
data = data.rename(columns={'Prediction':'rating'})

In [82]:
data.head()

,rating,userID,itemID
0,4,44,1
1,3,61,1
2,4,67,1
3,3,72,1
4,5,86,1


In [83]:
reader = Reader(rating_scale=(1, 5))
surp = Dataset.load_from_df(data[['userID', 'itemID', 'rating']], reader)

In [84]:
algo = SVD()
trainset = surp.build_full_trainset()
algo.fit(trainset)

In [95]:
test = pd.read_csv('examples_sample_submission.csv')

In [96]:
test['userID'] = test['Id'].apply(get_users)
test['itemID'] = test['Id'].apply(get_items)
test = test.drop('Id', axis=1)

In [97]:
a = test.apply(lambda row: round(algo.predict(row.userID, row.itemID).est) , axis=1)
test['Prediction'] = a

In [98]:
test['Id'] = 'r'+test['userID'].map(str)+'_'+'c'+test['itemID'].map(str)
test = test.drop(['userID','itemID'], axis=1)[['Id', 'Prediction']]

In [99]:
test.head()

,Id,Prediction
0,r37_c1,3.0
1,r73_c1,3.0
2,r156_c1,4.0
3,r160_c1,3.0
4,r248_c1,3.0


In [101]:
test.to_csv('subVince', index=False)